In [1]:
import os
from glob import glob # extract path of each file
import pandas as pd # data preprocessing
from xml.etree import ElementTree as et # parse information from XML
from functools import reduce

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
# step-1: get path of each xml file
xmlfiles = glob(r'C:/Users/prish/OneDrive/Desktop/yolo/1_datapreparation/data_images/*.xml')
# replace \\ with /
replace_text = lambda x: x.replace('\\','/')
xmlfiles = list(map(replace_text,xmlfiles))

In [11]:
xmlfiles

['C:/Users/prish/OneDrive/Desktop/yolo/1_datapreparation/data_images/000001.xml',
 'C:/Users/prish/OneDrive/Desktop/yolo/1_datapreparation/data_images/000002.xml',
 'C:/Users/prish/OneDrive/Desktop/yolo/1_datapreparation/data_images/000007.xml',
 'C:/Users/prish/OneDrive/Desktop/yolo/1_datapreparation/data_images/000009.xml',
 'C:/Users/prish/OneDrive/Desktop/yolo/1_datapreparation/data_images/000012.xml',
 'C:/Users/prish/OneDrive/Desktop/yolo/1_datapreparation/data_images/000016.xml',
 'C:/Users/prish/OneDrive/Desktop/yolo/1_datapreparation/data_images/000017.xml',
 'C:/Users/prish/OneDrive/Desktop/yolo/1_datapreparation/data_images/000019.xml',
 'C:/Users/prish/OneDrive/Desktop/yolo/1_datapreparation/data_images/000020.xml',
 'C:/Users/prish/OneDrive/Desktop/yolo/1_datapreparation/data_images/000021.xml',
 'C:/Users/prish/OneDrive/Desktop/yolo/1_datapreparation/data_images/000023.xml',
 'C:/Users/prish/OneDrive/Desktop/yolo/1_datapreparation/data_images/000024.xml',
 'C:/Users/prish

In [13]:
# step-2: read xml files
# from each xml file we need to extract
# filename, size(width, height), object(name, xmin, xmax, ymin, ymax)
def extract_text(filename):
    tree = et.parse(filename)
    root = tree.getroot()

    # extract filename
    image_name = root.find('filename').text
    # width and height of the image
    width = root.find('size').find('width').text
    height = root.find('size').find('height').text
    objs = root.findall('object')
    parser = []
    for obj in objs:
        name = obj.find('name').text
        bndbox = obj.find('bndbox')
        xmin = bndbox.find('xmin').text
        xmax = bndbox.find('xmax').text
        ymin = bndbox.find('ymin').text
        ymax = bndbox.find('ymax').text
        parser.append([image_name, width, height, name,xmin,xmax,ymin,ymax])
        
    return parser

In [15]:
parser_all = list(map(extract_text,xmlfiles))

In [16]:
data = reduce(lambda x, y : x+y,parser_all)

In [17]:
parser_all = list(map(extract_text,xmlfiles))

In [18]:
data = reduce(lambda x, y : x+y,parser_all)

In [19]:
df = pd.DataFrame(data,columns = ['filename','width','height','name','xmin','xmax','ymin','ymax'])

In [25]:
df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax
0,000001.jpg,1024,657,car,14,301,335,522
1,000001.jpg,1024,657,car,269,571,345,489
2,000001.jpg,1024,657,car,502,798,342,450
3,000001.jpg,1024,657,car,709,1009,333,438
4,000002.jpg,800,600,car,41,768,240,497


In [27]:
df.shape

(12055, 8)

In [29]:
df['name'].value_counts()

name
person         4182
car            1232
chair          1120
pottedplant     491
bird            487
bottle          477
dog             421
bicycle         335
sofa            331
horse           330
boat            321
cat             309
motorbike       289
tvmonitor       268
sheep           268
cow             258
train           254
diningtable     247
aeroplane       245
bus             190
Name: count, dtype: int64

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12055 entries, 0 to 12054
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  12055 non-null  object
 1   width     12055 non-null  object
 2   height    12055 non-null  object
 3   name      12055 non-null  object
 4   xmin      12055 non-null  object
 5   xmax      12055 non-null  object
 6   ymin      12055 non-null  object
 7   ymax      12055 non-null  object
dtypes: object(8)
memory usage: 753.6+ KB


In [33]:
# type conversion
cols = ['width','height','xmin','xmax','ymin','ymax']
df[cols] = df[cols].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12055 entries, 0 to 12054
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  12055 non-null  object
 1   width     12055 non-null  int32 
 2   height    12055 non-null  int32 
 3   name      12055 non-null  object
 4   xmin      12055 non-null  int32 
 5   xmax      12055 non-null  int32 
 6   ymin      12055 non-null  int32 
 7   ymax      12055 non-null  int32 
dtypes: int32(6), object(2)
memory usage: 471.0+ KB


In [35]:
# center x, center y
df['center_x'] = ((df['xmax']+df['xmin'])/2)/df['width']
df['center_y'] = ((df['ymax']+df['ymin'])/2)/df['height']
# w 
df['w'] = (df['xmax']-df['xmin'])/df['width']
# h 
df['h'] = (df['ymax']-df['ymin'])/df['height']

In [37]:
df.head()


,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
0,000001.jpg,1024,657,car,14,301,335,522,0.153809,0.652207,0.280273,0.284627
1,000001.jpg,1024,657,car,269,571,345,489,0.410156,0.634703,0.294922,0.219178
2,000001.jpg,1024,657,car,502,798,342,450,0.634766,0.602740,0.289062,0.164384
3,000001.jpg,1024,657,car,709,1009,333,438,0.838867,0.586758,0.292969,0.159817
4,000002.jpg,800,600,car,41,768,240,497,0.505625,0.614167,0.908750,0.428333


In [39]:
images = df['filename'].unique()

In [41]:
len(images)

3906

In [43]:
# 80% train and 20% test
img_df = pd.DataFrame(images,columns=['filename'])
img_train = tuple(img_df.sample(frac=0.8)['filename']) # shuffle and pick 80% of images

In [45]:
img_test = tuple(img_df.query(f'filename not in {img_train}')['filename']) # take rest 20% images

In [47]:
len(img_train), len(img_test)


(3125, 781)

In [49]:
train_df = df.query(f'filename in {img_train}')
test_df = df.query(f'filename in {img_test}')

In [51]:
train_df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
0,000001.jpg,1024,657,car,14,301,335,522,0.153809,0.652207,0.280273,0.284627
1,000001.jpg,1024,657,car,269,571,345,489,0.410156,0.634703,0.294922,0.219178
2,000001.jpg,1024,657,car,502,798,342,450,0.634766,0.602740,0.289062,0.164384
3,000001.jpg,1024,657,car,709,1009,333,438,0.838867,0.586758,0.292969,0.159817
4,000002.jpg,800,600,car,41,768,240,497,0.505625,0.614167,0.908750,0.428333


In [53]:
test_df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
7,000009.jpg,500,375,horse,69,270,172,330,0.339,0.669333,0.402,0.421333
8,000009.jpg,500,375,person,150,229,141,284,0.379,0.566667,0.158,0.381333
9,000009.jpg,500,375,person,285,327,201,331,0.612,0.709333,0.084,0.346667
10,000009.jpg,500,375,person,258,297,198,329,0.555,0.702667,0.078,0.349333
11,000012.jpg,500,333,car,156,351,97,270,0.507,0.551051,0.390,0.519520


In [55]:
# label encoding
def label_encoding(x):
    labels = {'person':0, 'car':1, 'chair':2, 'bottle':3, 'pottedplant':4, 'bird':5, 'dog':6,
       'sofa':7, 'bicycle':8, 'horse':9, 'boat':10, 'motorbike':11, 'cat':12, 'tvmonitor':13,
       'cow':14, 'sheep':15, 'aeroplane':16, 'train':17, 'diningtable':18, 'bus':19}
    return labels[x]

In [57]:
train_df['id'] = train_df['name'].apply(label_encoding)
test_df['id'] = test_df['name'].apply(label_encoding)

In [59]:
train_df.head(10)

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h,id
0,000001.jpg,1024,657,car,14,301,335,522,0.153809,0.652207,0.280273,0.284627,1
1,000001.jpg,1024,657,car,269,571,345,489,0.410156,0.634703,0.294922,0.219178,1
2,000001.jpg,1024,657,car,502,798,342,450,0.634766,0.602740,0.289062,0.164384,1
3,000001.jpg,1024,657,car,709,1009,333,438,0.838867,0.586758,0.292969,0.159817,1
4,000002.jpg,800,600,car,41,768,240,497,0.505625,0.614167,0.908750,0.428333,1
5,000002.jpg,800,600,car,533,722,236,299,0.784375,0.445833,0.236250,0.105000,1
6,000007.jpg,500,333,car,141,500,50,330,0.641000,0.570571,0.718000,0.840841,1
12,000016.jpg,334,500,bicycle,92,305,72,473,0.594311,0.545000,0.637725,0.802000,8
13,000017.jpg,480,364,person,185,279,62,199,0.483333,0.358516,0.195833,0.376374,0
14,000017.jpg,480,364,horse,90,403,78,336,0.513542,0.568681,0.652083,0.708791,9


In [61]:
import os
from shutil import move

In [65]:
current_directory = os.getcwd()
train_folder = 'data_images/train'
test_folder = 'data_images/test'


os.mkdir(train_folder)
os.mkdir(test_folder)

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'data_images/train'

In [67]:
current_directory


'C:\\Users\\prish\\OneDrive\\Desktop\\yolo\\1_datapreparation'

In [88]:
cols = ['filename','id','center_x','center_y', 'w', 'h']
groupby_obj_train = train_df[cols].groupby('filename')
groupby_obj_test = test_df[cols].groupby('filename')

In [90]:
#groupby_obj_train.get_group('000009.jpg').set_index('filename').to_csv('sample.txt',index=False,header=False)
# save each image in train/test folder and repective labels in .txt
def save_data(filename, folder_path, group_obj):
    # move image
    src = os.path.join('data_images',filename)
    dst = os.path.join(folder_path,filename)
    move(src,dst) # move image to the destination folder
    
    # save the labels
    text_filename = os.path.join(folder_path,
                                 os.path.splitext(filename)[0]+'.txt')
    group_obj.get_group(filename).set_index('filename').to_csv(text_filename,sep=' ',index=False,header=False)
    

In [93]:
filename_series = pd.Series(groupby_obj_train.groups.keys())

In [99]:
filename_series.apply(save_data,args=(train_folder,groupby_obj_train))

0       None
1       None
2       None
3       None
4       None
        ... 
3120    None
3121    None
3122    None
3123    None
3124    None
Length: 3125, dtype: object

In [109]:
filename_series_test = pd.Series(groupby_obj_test.groups.keys())
filename_series_test.apply(save_data,args=(test_folder,groupby_obj_test))

FileNotFoundError: [WinError 2] The system cannot find the file specified

True
